In [1]:
import pymongo 
import pandas as pd
import numpy as np
import config

In [2]:
# Connect with the portnumber and host  
client = pymongo.MongoClient(config.mongo_conn)  
    
# Access database  
cl_db = client['craigslist_db']  
    
# Access collection of the database  
cl_collection = cl_db['listings']  

In [3]:
cursor = cl_collection.find({},{ 'data_id': 1, 'listing_price': 1, 'listing_addrlocality': 1,'listing_addrzip': 1,'listing_bedbath': 1,'listing_type': 1, 'listing_datetime': 1,'listing_sqft': 1, '_id': 0 })   

list_cur = list(cursor)
df = pd.DataFrame(list_cur)
df["listing_price"] = df["listing_price"].str.replace("$","")
df["listing_price"] = df["listing_price"].str.replace(",","")
df["listing_sqft"] = df["listing_sqft"].str.replace("ft2","")


In [4]:
df.head()

,data_id,listing_price,listing_addrlocality,listing_addrzip,listing_bedbath,listing_type,listing_sqft,listing_datetime
0,7278244211,2095,Carlsbad,92009,1BR / 1Ba,Apartment,679,2021-02-23 15:37
1,7280806990,2920,San Diego,92121,2BR / 2Ba,Apartment,1098,2021-02-21 15:13
2,7278515816,1575,Escondido,92033,2BR / 2Ba,Apartment,948,2021-02-21 15:08
3,7277910674,2030,San Marcos,92079,1BR / 1Ba,Apartment,829,2021-02-21 15:08
4,7279719841,1437,Escondido,92033,1BR / 1Ba,Apartment,668,2021-02-21 14:57


In [9]:
df.count()

data_id                 9069
listing_price           9069
listing_addrlocality    8324
listing_addrzip         8324
listing_bedbath         8324
listing_type            8324
listing_sqft            8324
listing_datetime        7764
dtype: int64

In [10]:
blank_listing_sqft_df = df.loc[df['listing_sqft'] == '']
data_ids_blank_sqft = blank_listing_sqft_df['data_id']

# loop through blank sqft ids
for data_id in data_ids_blank_sqft:
    df.drop(df.index[df['data_id'] == data_id], inplace = True)


In [12]:
blank_zip_sqft_df = df.loc[df['listing_addrzip'] == '']
data_ids_blank_zip = blank_zip_sqft_df['data_id']

# loop through blank zip ids
for data_id in data_ids_blank_zip:
    df.drop(df.index[df['data_id'] == data_id], inplace = True)


In [13]:
clean_df = df.dropna()


In [15]:
clean_df

,data_id,listing_price,listing_addrlocality,listing_addrzip,listing_bedbath,listing_type,listing_sqft,listing_datetime
0,7278244211,2095,Carlsbad,92009,1BR / 1Ba,Apartment,679,2021-02-23 15:37
1,7280806990,2920,San Diego,92121,2BR / 2Ba,Apartment,1098,2021-02-21 15:13
2,7278515816,1575,Escondido,92033,2BR / 2Ba,Apartment,948,2021-02-21 15:08
3,7277910674,2030,San Marcos,92079,1BR / 1Ba,Apartment,829,2021-02-21 15:08
4,7279719841,1437,Escondido,92033,1BR / 1Ba,Apartment,668,2021-02-21 14:57
...,...,...,...,...,...,...,...,...
8145,7281372765,1785,Santee,92072,1BR / 1Ba,Apartment,700,2021-02-22 16:03
8146,7281378464,1605,Lemon Grove,91945,1BR / 1Ba,Apartment,616,2021-02-22 15:59
8147,7281369119,1508,El Cajon,92022,1BR / 1Ba,Apartment,740,2021-02-22 15:54
8148,7281369399,1508,El Cajon,92022,1BR / 1Ba,Apartment,740,2021-02-22 15:48


In [16]:
clean_df.dtypes 

data_id                 object
listing_price           object
listing_addrlocality    object
listing_addrzip         object
listing_bedbath         object
listing_type            object
listing_sqft            object
listing_datetime        object
dtype: object

In [19]:
#change type string to float so it's easier to get avg price& sqft later

clean_df["listing_price"] = clean_df["listing_price"].astype(float)
clean_df["listing_sqft"] = pd.to_numeric(clean_df['listing_sqft'] , downcast = "float" )
clean_df["listing_datetime"] = pd.to_datetime(clean_df['listing_datetime'])


In [20]:
clean_df.dtypes 

data_id                         object
listing_price                  float64
listing_addrlocality            object
listing_addrzip                 object
listing_bedbath                 object
listing_type                    object
listing_sqft                   float32
listing_datetime        datetime64[ns]
dtype: object

In [21]:
clean_df

,data_id,listing_price,listing_addrlocality,listing_addrzip,listing_bedbath,listing_type,listing_sqft,listing_datetime
0,7278244211,2095.0,Carlsbad,92009,1BR / 1Ba,Apartment,679.0,2021-02-23 15:37:00
1,7280806990,2920.0,San Diego,92121,2BR / 2Ba,Apartment,1098.0,2021-02-21 15:13:00
2,7278515816,1575.0,Escondido,92033,2BR / 2Ba,Apartment,948.0,2021-02-21 15:08:00
3,7277910674,2030.0,San Marcos,92079,1BR / 1Ba,Apartment,829.0,2021-02-21 15:08:00
4,7279719841,1437.0,Escondido,92033,1BR / 1Ba,Apartment,668.0,2021-02-21 14:57:00
...,...,...,...,...,...,...,...,...
8145,7281372765,1785.0,Santee,92072,1BR / 1Ba,Apartment,700.0,2021-02-22 16:03:00
8146,7281378464,1605.0,Lemon Grove,91945,1BR / 1Ba,Apartment,616.0,2021-02-22 15:59:00
8147,7281369119,1508.0,El Cajon,92022,1BR / 1Ba,Apartment,740.0,2021-02-22 15:54:00
8148,7281369399,1508.0,El Cajon,92022,1BR / 1Ba,Apartment,740.0,2021-02-22 15:48:00
